In [ ]:
from __future__ import division, print_function, absolute_import
import tflearn
import requests
import os
import zipfile
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.estimator import regression

In [1]:
# Parent folder where train and test images are saved
main_dir = '/home/ubuntu/pynb/kaggle-ndsb1/data'

def extract_data(filename, output_dir):
    '''
    Extracts zipped train and test files
    '''
    # Change working directory to the one containing train.zip & test.zip
    os.chdir(output_dir)
    
    # Extracts zip files
    print ("Extracting " + filename)
    
    fh = open(filename, 'rb')
    z = zipfile.ZipFile(fh)
    for name in z.namelist():
        z.extract(name, output_dir)
    fh.close()
    print ("Extracted " + filename)
        
extract_data("train.zip", main_dir)
extract_data("test.zip", main_dir)

Extracting train.zip
Extracted train.zip
Extracting test.zip
Extracted test.zip


In [1]:
# Build numpy array of images, resize to 48x48 and shuffle dataset
import cv2
import numpy as np
import os
import collections

main_dir = '/home/ubuntu/pynb/kaggle-ndsb1/data'
# Output folder where augmented images will be saved
aug_dir = '/home/ubuntu/pynb/kaggle-ndsb1/data/aug'

def img_aug(main_dir, aug_dir):
    '''
    Read images with OpenCV, perform augmentation and save the augmented images on disk
    '''
    # Create aug dir if does not exist
    if not os.path.exists(aug_dir):
        os.makedirs(aug_dir)
    print ("Initializing . . .")
    # Iterate over image classes
    for pclass in os.listdir(main_dir + "/train"):
        # Create aug/class dir if does not exist
        if not os.path.exists(aug_dir + "/" + pclass):
            os.makedirs(aug_dir + "/" + pclass)
        # Iterate over images in each class
        for pimg in os.listdir(main_dir + "/train/" + pclass):
            img = cv2.imread(main_dir + '/train/' + pclass + '/' + pimg)
            res = cv2.resize(img, (48, 48), interpolation=cv2.INTER_AREA)
            cv_gray = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
            rows,cols = cv_gray.shape
            # Save base resized and grayscaled image
            cv2.imwrite(aug_dir + "/" + pclass + "/" + pimg[:-4] + "_base.jpg", cv_gray)
            # Perform 3 rotations and save them to disk separately
            M1 = cv2.getRotationMatrix2D((cols/2,rows/2),90,1)
            dst1 = cv2.warpAffine(cv_gray,M1,(cols,rows))
            cv2.imwrite(aug_dir + "/" + pclass + "/" + pimg[:-4] + "_aug1.jpg", dst1)
            M2 = cv2.getRotationMatrix2D((cols/2,rows/2),180,1)
            dst2 = cv2.warpAffine(cv_gray,M2,(cols,rows))
            cv2.imwrite(aug_dir + "/" + pclass + "/" + pimg[:-4] + "_aug2.jpg", dst2)
            M3 = cv2.getRotationMatrix2D((cols/2,rows/2),270,1)
            dst3 = cv2.warpAffine(cv_gray,M3,(cols,rows))
            cv2.imwrite(aug_dir + "/" + pclass + "/" + pimg[:-4] + "_aug3.jpg", dst3)
    print ("Done data augmentation.")

img_aug(main_dir, aug_dir)

Initializing . . .
Done data augmentation.


In [2]:
import os
import cv2

aug_dir = '/home/ubuntu/pynb/kaggle-ndsb1/data/aug'

def img_to_dict(aug_dir):
    '''
    Read images with OpenCV, perform transformation and save in dict
    '''
    print ("Initializing . . .")
    # Initialize empty dict to contain each image as an array
    img_dict = {}
    # Iterate over image classes
    for pclass in os.listdir(aug_dir):
        # Iterate over images in a class
        for pimg in os.listdir(aug_dir + "/" + pclass):
            img = cv2.imread(aug_dir + '/' + pclass + '/' + pimg)
            cv_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img_dict[pimg] = {pclass: cv_gray}
    print ("Images sucessfully converted to dict.")
    return img_dict

img_dict = img_to_dict(aug_dir)
print len(img_dict)

Initializing . . .
Images sucessfully converted to dict.
121344


In [3]:
print img_dict['907_base.jpg']

{'acantharia_protist': array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ..., 
       [254, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 254, 254, ..., 255, 255, 255]], dtype=uint8)}


In [10]:
# Save dict
import pickle
with open('img_dict_2.pickle', 'wb') as handle:
    pickle.dump(img_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print ("File saved.")

File saved.


In [1]:
# Load dict
import pickle
with open('img_dict_2.pickle', 'rb') as handle:
    img_dict = pickle.load(handle)
print ("File loaded.")

File loaded.


In [4]:
import numpy as np

def prepare_dataset(img_dict):
    '''
    Constructs dataset of images as array and its labels.
    Also normalizes pixel values by mean and std deviation.
    '''
    labels = [] # Need to change labels to numpy array
    dataset = np.ndarray([len(img_dict),48,48,1], dtype='float32')
    i = 0
    # Iterate image and its class-array values pair
    for img, pclass_arr in img_dict.iteritems():
        
        labels.append(pclass_arr.keys()[0])
        im_array = pclass_arr.values()[0].reshape((48,48,1))
        mean = np.mean(im_array, dtype='float32')
        std = np.std(im_array, dtype='float32')
        # Apply normalization to each image
        im_array = (im_array - mean) / std 
        dataset[i,:,:,:] = im_array
        i += 1
        
    # Encode labels
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    
    return dataset, labels
    
dataset, labels = prepare_dataset(img_dict)
print (dataset.shape, labels.shape)

((121344, 48, 48, 1), (121344, 121))


In [5]:
print dataset[0]

[[[-6.74140358]
  [ 0.18542044]
  [ 0.18542044]
  ..., 
  [ 0.18542044]
  [ 0.18542044]
  [ 0.18542044]]

 [[-6.74140358]
  [ 0.18542044]
  [ 0.18542044]
  ..., 
  [ 0.18542044]
  [ 0.18542044]
  [ 0.18542044]]

 [[-6.74140358]
  [ 0.18542044]
  [ 0.18542044]
  ..., 
  [ 0.18542044]
  [ 0.18542044]
  [ 0.18542044]]

 ..., 
 [[-6.74140358]
  [ 0.18542044]
  [ 0.18542044]
  ..., 
  [ 0.18542044]
  [ 0.18542044]
  [ 0.18542044]]

 [[-6.74140358]
  [ 0.18542044]
  [ 0.18542044]
  ..., 
  [ 0.18542044]
  [ 0.18542044]
  [ 0.18542044]]

 [[-6.74140358]
  [ 0.18542044]
  [ 0.18542044]
  ..., 
  [ 0.18542044]
  [ 0.18542044]
  [ 0.18542044]]]


In [6]:
print labels[0]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]


In [3]:
# Save dataset and labels
import pickle
with open('dataset.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('labels.pickle', 'wb') as handle:
    pickle.dump(labels, handle, protocol=pickle.HIGHEST_PROTOCOL)
print ("File saved.")

File saved.


In [1]:
# Load dataset and labels
import pickle
with open('dataset.pickle', 'rb') as handle:
    img_dict = pickle.load(handle)
with open('labels.pickle', 'rb') as handle:
    img_dict = pickle.load(handle)
print ("File loaded.")

File loaded.


In [7]:
# Split training data to train and validation, then shuffle datasets
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=42)

for train_index, val_index in sss.split(dataset, labels):
    print("TRAIN:", train_index, "VAL:", val_index)
    X_train, X_val = dataset[train_index], dataset[val_index]
    y_train, y_val = labels[train_index], labels[val_index]

('TRAIN:', array([67139, 80980, 74225, ..., 12282, 35638, 11316]), 'VAL:', array([115058, 119818, 104462, ...,  85887,  69888,  18654]))
('TRAIN:', array([ 65522,  39899, 116935, ..., 114485,  48610,  97844]), 'VAL:', array([81710, 73998, 38975, ..., 29230, 60841, 73892]))
('TRAIN:', array([13844, 41144, 53608, ...,  4187, 87474, 81950]), 'VAL:', array([117790,  17974,  59888, ...,  18291,  39433,  94389]))


In [8]:
# Save dataset and labels
import pickle
with open('train.pickle', 'wb') as handle:
    pickle.dump([X_train, y_train], handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('val.pickle', 'wb') as handle:
    pickle.dump([X_val, y_val], handle, protocol=pickle.HIGHEST_PROTOCOL)
print ("File saved.")

File saved.


In [1]:
# Load dataset and labels
import pickle
with open('train.pickle', 'rb') as handle:
    X_train, y_train = pickle.load(handle)
with open('val.pickle', 'rb') as handle:
    X_val, y_val = pickle.load(handle)
print ("File loaded.")

File loaded.


In [6]:
# Build the preloader array, resize images to 48x48, and shuffle dataset
# Note: Error on importing preloader array since 1 Mar 2017. Thus, manual data aug was performed instead
'''from tflearn.data_utils import image_preloader
from tflearn.data_utils import shuffle
X, Y = image_preloader('data/train/', image_shape=(48, 48), mode='folder',
                       categorical_labels=True, grayscale=True, normalize=False)
X, Y = shuffle(X, Y)

print (X[0], Y[0])

# Preprocess images
img_prep = tflearn.ImagePreprocessing()
# Zero Center (With mean computed over the whole dataset)
img_prep.add_featurewise_zero_center()
# STD Normalization (With std computed over the whole dataset)
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_flip_updown()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=2.)'''

ImportError: cannot import name image_preloader

In [2]:
from __future__ import division, print_function, absolute_import
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.merge_ops import merge

# Model - Inception V3
network = input_data(shape=[None, 48, 48, 1])
conv1_7_7 = conv_2d(network, 64, 7, strides=2, activation='relu', name = 'conv1_7_7_s2')
pool1_3_3 = max_pool_2d(conv1_7_7, 3,strides=2)
pool1_3_3 = local_response_normalization(pool1_3_3)
conv2_3_3_reduce = conv_2d(pool1_3_3, 64,1, activation='relu',name = 'conv2_3_3_reduce')
conv2_3_3 = conv_2d(conv2_3_3_reduce, 192,3, activation='relu', name='conv2_3_3')
conv2_3_3 = local_response_normalization(conv2_3_3)
pool2_3_3 = max_pool_2d(conv2_3_3, kernel_size=3, strides=2, name='pool2_3_3_s2')
inception_3a_1_1 = conv_2d(pool2_3_3, 64, 1, activation='relu', name='inception_3a_1_1')
inception_3a_3_3_reduce = conv_2d(pool2_3_3, 96,1, activation='relu', name='inception_3a_3_3_reduce')
inception_3a_3_3 = conv_2d(inception_3a_3_3_reduce, 128,filter_size=3,  activation='relu', name = 'inception_3a_3_3')
inception_3a_5_5_reduce = conv_2d(pool2_3_3,16, filter_size=1,activation='relu', name ='inception_3a_5_5_reduce' )
inception_3a_5_5 = conv_2d(inception_3a_5_5_reduce, 32, filter_size=5, activation='relu', name= 'inception_3a_5_5')
inception_3a_pool = max_pool_2d(pool2_3_3, kernel_size=3, strides=1, )
inception_3a_pool_1_1 = conv_2d(inception_3a_pool, 32, filter_size=1, activation='relu', name='inception_3a_pool_1_1')

# merge the inception_3a__
inception_3a_output = merge([inception_3a_1_1, inception_3a_3_3, inception_3a_5_5, inception_3a_pool_1_1], mode='concat', axis=3)

inception_3b_1_1 = conv_2d(inception_3a_output, 128,filter_size=1,activation='relu', name= 'inception_3b_1_1' )
inception_3b_3_3_reduce = conv_2d(inception_3a_output, 128, filter_size=1, activation='relu', name='inception_3b_3_3_reduce')
inception_3b_3_3 = conv_2d(inception_3b_3_3_reduce, 192, filter_size=3,  activation='relu',name='inception_3b_3_3')
inception_3b_5_5_reduce = conv_2d(inception_3a_output, 32, filter_size=1, activation='relu', name = 'inception_3b_5_5_reduce')
inception_3b_5_5 = conv_2d(inception_3b_5_5_reduce, 96, filter_size=5,  name = 'inception_3b_5_5')
inception_3b_pool = max_pool_2d(inception_3a_output, kernel_size=3, strides=1,  name='inception_3b_pool')
inception_3b_pool_1_1 = conv_2d(inception_3b_pool, 64, filter_size=1,activation='relu', name='inception_3b_pool_1_1')

#merge the inception_3b_*
inception_3b_output = merge([inception_3b_1_1, inception_3b_3_3, inception_3b_5_5, inception_3b_pool_1_1], mode='concat',axis=3,name='inception_3b_output')

pool3_3_3 = max_pool_2d(inception_3b_output, kernel_size=3, strides=2, name='pool3_3_3')
inception_4a_1_1 = conv_2d(pool3_3_3, 192, filter_size=1, activation='relu', name='inception_4a_1_1')
inception_4a_3_3_reduce = conv_2d(pool3_3_3, 96, filter_size=1, activation='relu', name='inception_4a_3_3_reduce')
inception_4a_3_3 = conv_2d(inception_4a_3_3_reduce, 208, filter_size=3,  activation='relu', name='inception_4a_3_3')
inception_4a_5_5_reduce = conv_2d(pool3_3_3, 16, filter_size=1, activation='relu', name='inception_4a_5_5_reduce')
inception_4a_5_5 = conv_2d(inception_4a_5_5_reduce, 48, filter_size=5,  activation='relu', name='inception_4a_5_5')
inception_4a_pool = max_pool_2d(pool3_3_3, kernel_size=3, strides=1,  name='inception_4a_pool')
inception_4a_pool_1_1 = conv_2d(inception_4a_pool, 64, filter_size=1, activation='relu', name='inception_4a_pool_1_1')

inception_4a_output = merge([inception_4a_1_1, inception_4a_3_3, inception_4a_5_5, inception_4a_pool_1_1], mode='concat', axis=3, name='inception_4a_output')


inception_4b_1_1 = conv_2d(inception_4a_output, 160, filter_size=1, activation='relu', name='inception_4a_1_1')
inception_4b_3_3_reduce = conv_2d(inception_4a_output, 112, filter_size=1, activation='relu', name='inception_4b_3_3_reduce')
inception_4b_3_3 = conv_2d(inception_4b_3_3_reduce, 224, filter_size=3, activation='relu', name='inception_4b_3_3')
inception_4b_5_5_reduce = conv_2d(inception_4a_output, 24, filter_size=1, activation='relu', name='inception_4b_5_5_reduce')
inception_4b_5_5 = conv_2d(inception_4b_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4b_5_5')

inception_4b_pool = max_pool_2d(inception_4a_output, kernel_size=3, strides=1,  name='inception_4b_pool')
inception_4b_pool_1_1 = conv_2d(inception_4b_pool, 64, filter_size=1, activation='relu', name='inception_4b_pool_1_1')

inception_4b_output = merge([inception_4b_1_1, inception_4b_3_3, inception_4b_5_5, inception_4b_pool_1_1], mode='concat', axis=3, name='inception_4b_output')


inception_4c_1_1 = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu',name='inception_4c_1_1')
inception_4c_3_3_reduce = conv_2d(inception_4b_output, 128, filter_size=1, activation='relu', name='inception_4c_3_3_reduce')
inception_4c_3_3 = conv_2d(inception_4c_3_3_reduce, 256,  filter_size=3, activation='relu', name='inception_4c_3_3')
inception_4c_5_5_reduce = conv_2d(inception_4b_output, 24, filter_size=1, activation='relu', name='inception_4c_5_5_reduce')
inception_4c_5_5 = conv_2d(inception_4c_5_5_reduce, 64,  filter_size=5, activation='relu', name='inception_4c_5_5')

inception_4c_pool = max_pool_2d(inception_4b_output, kernel_size=3, strides=1)
inception_4c_pool_1_1 = conv_2d(inception_4c_pool, 64, filter_size=1, activation='relu', name='inception_4c_pool_1_1')

inception_4c_output = merge([inception_4c_1_1, inception_4c_3_3, inception_4c_5_5, inception_4c_pool_1_1], mode='concat', axis=3,name='inception_4c_output')

inception_4d_1_1 = conv_2d(inception_4c_output, 112, filter_size=1, activation='relu', name='inception_4d_1_1')
inception_4d_3_3_reduce = conv_2d(inception_4c_output, 144, filter_size=1, activation='relu', name='inception_4d_3_3_reduce')
inception_4d_3_3 = conv_2d(inception_4d_3_3_reduce, 288, filter_size=3, activation='relu', name='inception_4d_3_3')
inception_4d_5_5_reduce = conv_2d(inception_4c_output, 32, filter_size=1, activation='relu', name='inception_4d_5_5_reduce')
inception_4d_5_5 = conv_2d(inception_4d_5_5_reduce, 64, filter_size=5,  activation='relu', name='inception_4d_5_5')
inception_4d_pool = max_pool_2d(inception_4c_output, kernel_size=3, strides=1,  name='inception_4d_pool')
inception_4d_pool_1_1 = conv_2d(inception_4d_pool, 64, filter_size=1, activation='relu', name='inception_4d_pool_1_1')

inception_4d_output = merge([inception_4d_1_1, inception_4d_3_3, inception_4d_5_5, inception_4d_pool_1_1], mode='concat', axis=3, name='inception_4d_output')

inception_4e_1_1 = conv_2d(inception_4d_output, 256, filter_size=1, activation='relu', name='inception_4e_1_1')
inception_4e_3_3_reduce = conv_2d(inception_4d_output, 160, filter_size=1, activation='relu', name='inception_4e_3_3_reduce')
inception_4e_3_3 = conv_2d(inception_4e_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_4e_3_3')
inception_4e_5_5_reduce = conv_2d(inception_4d_output, 32, filter_size=1, activation='relu', name='inception_4e_5_5_reduce')
inception_4e_5_5 = conv_2d(inception_4e_5_5_reduce, 128,  filter_size=5, activation='relu', name='inception_4e_5_5')
inception_4e_pool = max_pool_2d(inception_4d_output, kernel_size=3, strides=1,  name='inception_4e_pool')
inception_4e_pool_1_1 = conv_2d(inception_4e_pool, 128, filter_size=1, activation='relu', name='inception_4e_pool_1_1')


inception_4e_output = merge([inception_4e_1_1, inception_4e_3_3, inception_4e_5_5,inception_4e_pool_1_1],axis=3, mode='concat')

pool4_3_3 = max_pool_2d(inception_4e_output, kernel_size=3, strides=2, name='pool_3_3')


inception_5a_1_1 = conv_2d(pool4_3_3, 256, filter_size=1, activation='relu', name='inception_5a_1_1')
inception_5a_3_3_reduce = conv_2d(pool4_3_3, 160, filter_size=1, activation='relu', name='inception_5a_3_3_reduce')
inception_5a_3_3 = conv_2d(inception_5a_3_3_reduce, 320, filter_size=3, activation='relu', name='inception_5a_3_3')
inception_5a_5_5_reduce = conv_2d(pool4_3_3, 32, filter_size=1, activation='relu', name='inception_5a_5_5_reduce')
inception_5a_5_5 = conv_2d(inception_5a_5_5_reduce, 128, filter_size=5,  activation='relu', name='inception_5a_5_5')
inception_5a_pool = max_pool_2d(pool4_3_3, kernel_size=3, strides=1,  name='inception_5a_pool')
inception_5a_pool_1_1 = conv_2d(inception_5a_pool, 128, filter_size=1,activation='relu', name='inception_5a_pool_1_1')

inception_5a_output = merge([inception_5a_1_1, inception_5a_3_3, inception_5a_5_5, inception_5a_pool_1_1], axis=3,mode='concat')


inception_5b_1_1 = conv_2d(inception_5a_output, 384, filter_size=1,activation='relu', name='inception_5b_1_1')
inception_5b_3_3_reduce = conv_2d(inception_5a_output, 192, filter_size=1, activation='relu', name='inception_5b_3_3_reduce')
inception_5b_3_3 = conv_2d(inception_5b_3_3_reduce, 384,  filter_size=3,activation='relu', name='inception_5b_3_3')
inception_5b_5_5_reduce = conv_2d(inception_5a_output, 48, filter_size=1, activation='relu', name='inception_5b_5_5_reduce')
inception_5b_5_5 = conv_2d(inception_5b_5_5_reduce,128, filter_size=5,  activation='relu', name='inception_5b_5_5' )
inception_5b_pool = max_pool_2d(inception_5a_output, kernel_size=3, strides=1,  name='inception_5b_pool')
inception_5b_pool_1_1 = conv_2d(inception_5b_pool, 128, filter_size=1, activation='relu', name='inception_5b_pool_1_1')
inception_5b_output = merge([inception_5b_1_1, inception_5b_3_3, inception_5b_5_5, inception_5b_pool_1_1], axis=3, mode='concat')

pool5_7_7 = avg_pool_2d(inception_5b_output, kernel_size=7, strides=1)
pool5_7_7 = dropout(pool5_7_7, 0.4)
loss = fully_connected(pool5_7_7, 121,activation='softmax')

network = regression(loss, optimizer='momentum',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Training
model = tflearn.DNN(network, checkpoint_path='model_inception.ckpt',
                max_checkpoints=1, tensorboard_verbose=0)

model.fit(X_train, y_train, n_epoch=40, shuffle=False,
      show_metric=True, batch_size=32, snapshot_step=500,
      validation_set=(X_val, y_val), snapshot_epoch=False, run_id='vgg_plankton121_7')

# Save model
model.save('inception_1.tfl')
print ("Model saved.")

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to tf.summary.merge.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for inception_1.tfl
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_1/Const_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]
	 [[Node: save_1/RestoreV2_53/_155 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_982_save_1/RestoreV2_53", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'save_1/RestoreV2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py", line 3, in <module>
    app.launch_new_instanc